# **_Covid 19 Data Exploration._**

_((Skills used: Joins, CTE's, Temp Tables, Windows Functions, Aggregate Functions, Creating Views, Converting Data Types))_

**_Selecting a sample of Data that we are going to be starting with._**

In [ ]:
SELECT top 5 Location, date, total_cases, new_cases, total_deaths, population
From protofolioproject..CovidDeaths$
Where continent is not null and total_deaths is not NULL


## **Total Cases vs Total Deaths**  

- **Shows likelihood of dying if you contract covid in your country**

In [ ]:
select location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 as Death_Percentage
from protofolioproject..CovidDeaths$
where location like '%states%'
order by 1,2

## **_Total Cases vs Population_**

### 

- _**Shows what percentage of population infected with Covid**_

In [ ]:
select location, date,total_cases, population,(total_cases/population)*100 as infection_rate
from protofolioproject..CovidDeaths$
where location like '%states%'
order by 1,2,3

### 

- **_Countries with Highest Infection Rate compared to Population_**

In [ ]:
Select Location, Population, MAX(total_cases) as HighestInfectionCount,  Max((total_cases/population))*100 as PercentPopulationInfected
from protofolioproject..CovidDeaths$
Group by Location, Population
order by PercentPopulationInfected desc


- **_Countries with Highest Death Count per Population_**

In [ ]:
Select Location, MAX(cast(Total_deaths as int)) as TotalDeathCount
from protofolioproject..CovidDeaths$
Where continent is not null 
Group by Location
order by TotalDeathCount desc

## **_BREAKING THINGS DOWN BY CONTINENT!!!_**

### 

- **_Showing contintents with the highest death count per population_**

In [ ]:
Select continent, MAX(cast(Total_deaths as int)) as TotalDeathCount
from protofolioproject..CovidDeaths$
Where continent is not null 
Group by continent
order by TotalDeathCount desc

## **_Global Numbers._**

In [ ]:
Select SUM(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths, SUM(cast(new_deaths as int))/SUM(New_Cases)*100 as DeathPercentage
from protofolioproject..CovidDeaths$
where continent is not null 


## **_Total Population vs Vaccinations_**

- **_sample of the data from joining two tables first._**

In [ ]:
SELECT top 5 *
from protofolioproject..CovidDeaths$    dea
JOIN protofolioproject..CovidVaccinations$      vac
on  dea.[location]= vac.[location]
and dea.[date]= vac.date

- **_Shows Percentage of Population that has recieved at least one Covid Vaccine_**

    - **_we need to sum all the new\_vaccib=nation per location as it is not in the table below_**

In [ ]:
SELECT top 10 dea.continent, DEA.location, dea.[date], dea.population,  vac.new_vaccinations
FROM  protofolioproject..CovidDeaths$ dea
join protofolioproject..CovidVaccinations$ vac 
on dea.[location]=vac.[location]
and dea.[date]=vac.[date]
WHERE dea.continent is not NULL
order by 5 DESC


- **Total cumulative vaccinated**

In [ ]:
SELECT dea.continent, DEA.location, dea.[date], dea.population,  vac.new_vaccinations, SUM (cast (vac.new_vaccinations   as int)) 
    OVER  ( partition by  dea.location order by dea.location , dea.date) as cumulative_vac
FROM  protofolioproject..CovidDeaths$ dea
join protofolioproject..CovidVaccinations$ vac 
on dea.[location]=vac.[location]
and dea.[date]=vac.[date]
WHERE dea.continent is not NULL
order by 2,3


- **_Using CTE to perform Calculation on Partition By in previous query_**

In [ ]:
With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, cumulative_vac)
as
(
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as cumulative_vac
FROM  protofolioproject..CovidDeaths$ dea
join protofolioproject..CovidVaccinations$ vac 
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 
)
Select *, (cumulative_vac/Population)*100
From PopvsVac

- _**Using Temp Table to perform Calculation on Partition By in previous query**_

In [ ]:
DROP Table if exists #PercentPopulationVaccinated
Create Table #PercentPopulationVaccinated
(
Continent nvarchar(255),
Location nvarchar(255),
Date datetime,
Population numeric,
New_vaccinations numeric,
cumulative_vac int
)

Insert into #PercentPopulationVaccinated
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as cumulative_vac
--, (RollingPeopleVaccinated/population)*100
FROM  protofolioproject..CovidDeaths$ dea
join protofolioproject..CovidVaccinations$ vac 
	On dea.location = vac.location
	and dea.date = vac.date
--where dea.continent is not null 
--order by 2,3

Select * ,( cumulative_vac /Population)*100 as per_vac
From #PercentPopulationVaccinated

###  **_<u>Creating View to store data for later visualizations</u>_**

In [3]:
Create View PercentPopulationVaccinated as
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
FROM  protofolioproject..CovidDeaths$ dea
join protofolioproject..CovidVaccinations$ vac 
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 

: Msg 2714, Level 16, State 3, Procedure PercentPopulationVaccinated, Line 1
There is already an object named 'PercentPopulationVaccinated' in the database.

Total execution time: 00:00:00.005